# Greedy Algorithms

Lab associated with Module 8: Greedy Algorithms

***

In [71]:
# The following lines are used to increase the width of cells to utilize more space on the screen 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

***

### Section 0: Imports

In [72]:
import numpy as np

In [73]:
import math

In [74]:
from IPython.display import Image
from graphviz import Digraph
import heapq
import time
import random

***

### <font color='red'> Activity 1: Write code for creating a prefix tree for any arbitrary distribution, e.g., [A:45, B:13, C:12, D:16, E:9, F:5]. Your algorithm should return the prefix tree and should display the correct code for every alphabet.. </font>

In [75]:
#### TODO ####
### Good Luck ###

class Node:
    def __init__(self, char=None, freq=0):
        self.char = char      
        self.freq = freq      
        self.left = None      
        self.right = None     

    def __lt__(self, other):  # Sort by frequency when comparing heaps
        return self.freq < other.freq

# Constructing a Huffman tree
def build_tree(freq_dict):
    # 1. Put all characters into the root pile
    heap = [Node(ch, f) for ch, f in freq_dict.items()]
    heapq.heapify(heap)

    # 2. Take the two smallest ones each time
    while len(heap) > 1:
        n1 = heapq.heappop(heap)
        n2 = heapq.heappop(heap)
        parent = Node(freq=n1.freq + n2.freq)
        parent.left = n1
        parent.right = n2
        heapq.heappush(heap, parent)

    return heap[0]  

# Traversing the tree
def get_codes(node, code="", code_dict=None):
    if code_dict is None:
        code_dict = {}
    if node:
        if node.char is not None:  # Leaf nodes
            code_dict[node.char] = code
        get_codes(node.left, code + "0", code_dict)
        get_codes(node.right, code + "1", code_dict)
    return code_dict




In [76]:
example = {'A':45, 'B':13, 'C':12, 'D':16, 'E':9, 'F':5}
root = build_tree(example)
codes = get_codes(root)

print("Huffman：")
for ch, code in codes.items():
    print(ch, ":", code)

Huffman：
A : 0
C : 100
B : 101
F : 1100
E : 1101
D : 111


***

###  Prim's Algorithm

Graph's Preliminaries

In [77]:
from graph import *

In [78]:
G = Graph()

for i in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']:
    G.addVertex( Node(i) )

In [79]:
V = G.vertices

#0, 1, 2, 3, 4, 5, 6, 7, 8
#A, B, C, D, E, F, G, H, I

G.addBiEdge( V[0], V[1], 4)

G.addBiEdge( V[0], V[7] , 8)

G.addBiEdge( V[1], V[7], 11)

G.addBiEdge( V[1], V[2], 8)

G.addBiEdge( V[2], V[3], 7)

G.addBiEdge( V[3], V[4], 9)

G.addBiEdge( V[3], V[5], 14 )

G.addBiEdge( V[4], V[5], 10 )

G.addBiEdge( V[2], V[5], 4 )

G.addBiEdge( V[2], V[8], 2 )

G.addBiEdge( V[5], V[6], 2 )

G.addBiEdge( V[6], V[7], 1 )

G.addBiEdge( V[6], V[8], 6 )

G.addBiEdge( V[7], V[8], 7 )


In [80]:
print(G)

Graph with:
	 Vertices:
	A,B,C,D,E,F,G,H,I,
	 Edges:
	(A,(<graph.Node object at 0x0000021A45D55D30>, 4)) (A,(<graph.Node object at 0x0000021A447BA940>, 8)) (B,(<graph.Node object at 0x0000021A45D55C40>, 4)) (B,(<graph.Node object at 0x0000021A447BA940>, 11)) (B,(<graph.Node object at 0x0000021A45D55BE0>, 8)) (C,(<graph.Node object at 0x0000021A45D55D30>, 8)) (C,(<graph.Node object at 0x0000021A45D55280>, 7)) (C,(<graph.Node object at 0x0000021A447BA9D0>, 4)) (C,(<graph.Node object at 0x0000021A447BAE20>, 2)) (D,(<graph.Node object at 0x0000021A45D55BE0>, 7)) (D,(<graph.Node object at 0x0000021A447BAF10>, 9)) (D,(<graph.Node object at 0x0000021A447BA9D0>, 14)) (E,(<graph.Node object at 0x0000021A45D55280>, 9)) (E,(<graph.Node object at 0x0000021A447BA9D0>, 10)) (F,(<graph.Node object at 0x0000021A45D55280>, 14)) (F,(<graph.Node object at 0x0000021A447BAF10>, 10)) (F,(<graph.Node object at 0x0000021A45D55BE0>, 4)) (F,(<graph.Node object at 0x0000021A447BAC70>, 2)) (G,(<graph.Node object 

This is what we had in the lectures as the slow implementation of Prim's Algorithm

In [81]:
# G is graph
# s is the node to start

def slowPrim(G, s):
    
    # first, find the lightest edge leaving s
    bestWt = np.inf
    bestu = None
    
    for u,wt in s.getOutNeighborsWithWeights():
        
        if wt < bestWt:
            bestWt = wt
            bestu = u
    
    MST = [ (s, bestu) ]
    verticesVisited = [s,bestu]
    
    while len(verticesVisited) < len(G.vertices): # danger! this will loop forever if the graph isn't connected...
        
        # find the lightest edge (x,v) so that x has been visited and v hasn't.
        bestWt = np.inf
        bestv = None
        bestx = None
        
        for x in verticesVisited:
            for v,wt in x.getOutNeighborsWithWeights():     
                if v in verticesVisited:
                    continue
                
                if wt < bestWt:
                    bestWt = wt
                    bestv = v
                    bestx = x
                    
        MST.append((bestx,bestv))
        verticesVisited.append(bestv)
    
    return MST

In [82]:
T = slowPrim(G, G.vertices[0])

for x,y in T:
    print(x,y)
    


A B
A H
H G
G F
F C
C I
C D
D E


Okay, it seems to be working fine, but as we discussed, will be quite slow. Let us see if we can work on the faster version of the code as:

### <font color='red'> Activity 2: In lights of Prim's Algorithm above, write an efficient implementation based on our discussions in the Seminar/Lecture. </font>

In [83]:
def prim(G, w):
    #### TODO ####
    ### Good Luck ###
    
    # A counter to ensure that the priority queue does not try to compare
    counter = 0
    
    MST = []  
    visited = set([w])  
    edges = []  
    
    # Add all outgoing edges of the starting node to the priority queue
    for neighbor, weight in w.getOutNeighborsWithWeights():
        heapq.heappush(edges, (weight, counter, w, neighbor))
        counter += 1
    
    # When there are edges to process and not all nodes have been visited
    while edges and len(visited) < len(G.vertices):
        # Get the edge with the smallest weight
        weight, _, u, v = heapq.heappop(edges)
        
        # If the target node has not been visited
        if v not in visited:
            MST.append((u, v))
            visited.add(v)
            
            # Add all outgoing edges of the newly visited node to the priority queue
            for neighbor, w in v.getOutNeighborsWithWeights():
                if neighbor not in visited:
                    heapq.heappush(edges, (w, counter, v, neighbor))
                    counter += 1
    
    return MST

In [84]:
T = prim(G, G.vertices[0])

for x,y in T:
    print(x,y)

A B
A H
H G
G F
F C
C I
C D
D E


In [88]:
def generate_random_graph(num_vertices, edge_probability=0.3, max_weight=100):
    G = Graph()
    
    for i in range(num_vertices):
        G.addVertex(Node(str(i)))
    
    V = G.vertices
    
    for i in range(num_vertices):
        for j in range(i + 1, num_vertices):
            if random.random() < edge_probability:
                weight = random.randint(1, max_weight)
                G.addBiEdge(V[i], V[j], weight)
    
    return G
def simple_performance_comparison():
    graph_sizes = [10, 20, 50, 100, 200]
    
    print("Prim's algorithm")
    print("Vertices|First algorithm time |Second algorithm time | Speedup")
    
    for size in graph_sizes:

        G = generate_random_graph(size)
        
        start_time = time.time()
        mst_slow = slowPrim(G, G.vertices[0])
        time_slow = time.time() - start_time
        
        start_time = time.time()
        mst_fast = prim(G, G.vertices[0])
        time_fast = time.time() - start_time
        
        # Computational speedup
        speedup = time_slow / time_fast if time_fast > 0 else float('inf')
        
        print(f"{size:6} | {time_slow:15.6f} | {time_fast:15.6f} | {speedup:8.2f}x")

simple_performance_comparison()

Prim's algorithm
Vertices|First algorithm time |Second algorithm time | Speedup
    10 |        0.000000 |        0.000000 |      infx
    20 |        0.000000 |        0.000000 |      infx
    50 |        0.004000 |        0.000000 |      infx
   100 |        0.053209 |        0.001000 |    53.21x
   200 |        0.830225 |        0.004002 |   207.44x


***

# Task 4

In [23]:
def greedy_coin_change(coins, amount):
    coins.sort(reverse=True)  # Sort from largest to smallest
    result = []
    for coin in coins:
        while amount >= coin:
            result.append(coin)
            amount -= coin
    return result if amount == 0 else -1

def dp_coin_change(coins, amount):
    dp = [float("inf")] * (amount + 1)
    dp[0] = 0
    choice = [[] for _ in range(amount+1)]
    
    for i in range(1, amount+1):
        for c in coins:
            if i - c >= 0 and dp[i-c] + 1 < dp[i]:
                dp[i] = dp[i-c] + 1
                choice[i] = choice[i-c] + [c]
    return choice[amount] if dp[amount] != [] else -1


In [87]:
coins = [1,5,10,25]
amounts = [30, 37, 43]

for amt in amounts:
    print(f"\nAmount {amt}:")
    print("Greedy:", greedy_coin_change(coins, amt))
    print("DP    :", dp_coin_change(coins, amt))



Amount 30:
Greedy: [25, 5]
DP    : [5, 25]

Amount 37:
Greedy: [25, 10, 1, 1]
DP    : [1, 1, 10, 25]

Amount 43:
Greedy: [25, 10, 5, 1, 1, 1]
DP    : [1, 1, 1, 5, 10, 25]
